# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl")
include("lattice.jl")
include("basis.jl")
include("sparseHermitian.jl")
include("sparseHam.jl")
include("sparseS2.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 4
const Ly = 4
# number of sites
const N = Lx*Ly
# NN coupling
const J1 = 1.0
# NNN coupling
const J2 = 0.4
# plaquette coupling
const K = 0.3

# choose Sz sector
const Sz = 0
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0
const my = 0
const kx = 2*pi*mx/Lx
const ky = 2*pi*my/Ly
# choose spin inversion quantum number
const z = 1

# number of eigenvalues desired
const numEigs = 4
# a tolerance for error
const tolerance = 10.^(-8.)
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = true
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 40
# maximum number of iterations to converge eigenvalues
const maxIter = 200

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(Sz,kx,ky,z);

In [5]:
# construct the basis
# 0.34 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = reducedBasis{UInt64}(l,s)
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.152352 seconds (164.93 k allocations: 7.699 MB, 5.78% gc time)
Dimension of reduced Hilbert space is 441.


In [6]:
(sizeof(basis.b)+sizeof(basis.n))/10^9

7.056e-6

In [18]:
# basis memory and time
# N=16 0.0000071 GB, 0.0034 s
# N=20 0.000074 GB, 0.052 s
# N=24 0.00091 GB, 0.75 s
# N=30 0.041 GB, 43 s
# N=32 ~ 0.15 GB, 2.9 m (extrapolated)
# N=36 ~ 1.7 GB, 45 m (extrapolated)

In [8]:
# using ProfileView

In [9]:
# Profile.clear()
# @profile SzkxkyBasis(l,s)
# ProfileView.view()

#### build the Hamiltonian

In [10]:
# couplings type to make passing J1,K easier
const c = couplings(J1,J2,K)

# build the sparse Hamiltonian
# takes 0.95 seconds for 6x4 Heisenberg model, H contains ~0.013 GB
@time const H = constructSparseHam(basis,c,s,l);

  0.619155 seconds (663.35 k allocations: 23.633 MB, 1.38% gc time)


In [11]:
println("H: ",(sizeof(H.colPntrs)+sizeof(H.rowIndcs)+sizeof(H.nzVals)+sizeof(H.nzPntrs))/10^9," GB of memory.")

H: 8.4584e-5 GB of memory.


In [35]:
# Hamiltonian memory and time
# N=16 0.000084 GB, 0.0057 s
# N=20 0.0010 GB, 0.093 s
# N=24 0.015 GB, 1.2 s
# N=30 0.81 GB, 79 s
# N=32 ~ 2.9 GB, 5.3 m (extrapolated)
# N=36 ~ 45 GB, 1.4 h (extrapolated)

In [12]:
# using ProfileView

In [13]:
# Profile.clear()
# @profile constructSparseHam(basis,c,s,l)
# ProfileView.view()

#### construct on the fly $H |\psi \rangle$ operator

In [14]:
# using LinearMaps

In [15]:
# H_psi!(y::AbstractVector,x::AbstractVector) = H_mul_psi!(basis,c,s,l,y,x)

In [16]:
# HLM = LinearMap(H_psi!, basis.dim, Complex128, ismutating=true, ishermitian=true)

#### find eigenvalues and eigenvectors

In [17]:
# compute eigenvalues
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec);

  1.821323 seconds (2.18 M allocations: 85.549 MB, 1.52% gc time)


In [44]:
# Lanczos time
# N=16 0.037 s
# N=20 0.41 s
# N=24 6.2 s
# N=30 6.6 m -- ?18 m? (seemed to take ~2GB memory looking at activity monitor, time might be slow due to memory limitations of laptop)
# N=32 ~26 m (extrap.)
# N=36 ~6.9 h (extrap.)

In [19]:
# using ProfileView

In [20]:
# Profile.clear()
# @profile eigs(HLM; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:SR, tol=tolerance, ritzvec=ritzVec)
# ProfileView.view()

In [21]:
# print energies
println("Energies are: ")
for en in real(eigsResult[1])
    println(en)
end
# print algorithm performance
println("Number of iterations = ",eigsResult[3])
println("Number of matrix-vector multiplications = ",eigsResult[4])

Energies are: 
-6.994896080039556
-6.735843696892471
-6.73079053979335
-6.208232626070167
Number of iterations = 4
Number of matrix-vector multiplications = 3


In [22]:
# # if we can check vs the dense diagonalization, do so
# if N <= 16
#     const Hdense = constructDenseHam(basis,c,s,l)
#     ens = sort!(real(eigvals(Hdense)))
#     println("Energies are: ")
#     for en in ens[1:2*numEigs]
#         println(en)
#     end
# end

#### compute $S^2$ values and save spectral data
Save in format |  E  |  S(S+1)  |  Sz  |  mx  |  my  |  z  |

In [26]:
using DataFrames

In [27]:
# energies
EData = real(eigsResult[1])
# Sz values
SzData = fill(Sz,numEigs)
# mx values
mxData = fill(mx,numEigs)
# my values
myData = fill(my,numEigs)
# z values
zData = fill(z,numEigs);

In [28]:
# S(S+1) values
@time S2Data = round(Int64,real(S2expectations(basis,s,l,eigsResult[2])));

  0.195934 seconds (154.56 k allocations: 5.573 MB)


In [50]:
# total spin calculation time
# N=16 0.0056 s
# N=20 0.14 s
# N=24 2.5 s
# N=30 4.6 m
# N=32 ~ m (extrap.)
# N=36 ~9.8 h (extrap.)

In [30]:
# create DataFrame
df = DataFrame(E=EData,Ssqrd=S2Data,Sz=SzData,mx=mxData,my=myData,z=zData)

,E,Ssqrd,Sz,mx,my,z
1,-6.994896080039556,0,0,0,0,1
2,-6.735843696892471,0,0,0,0,1
3,-6.73079053979335,0,0,0,0,1
4,-6.208232626070167,6,0,0,0,1
